# 02 RNN 从零实现

> **核心问题**: 如何用 NumPy 从零实现一个完整的循环神经网络？

---

## 📚 学习目标

完成本章后，你将能够：

- [ ] 推导 RNN 的数学公式（隐状态更新、输出计算）
- [ ] 用 NumPy 实现 RNNCell（单个 RNN 单元）
- [ ] 用 NumPy 实现完整的 RNN（处理整个序列）
- [ ] 理解 RNN 参数的维度和含义
- [ ] 与 PyTorch 的 nn.RNN 对比验证

---

## 🔍 前置知识

请确保已完成：
- ✅ 01_why_rnn.ipynb（RNN 核心思想）
- ✅ 神经网络基础（前向传播、矩阵运算）

In [ ]:
# ============================================================
# 环境准备
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# 中文显示配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 💡 Part 1: RNN 数学公式

### 1.1 核心公式

RNN 在每个时间步 $t$ 执行以下计算：

**隐状态更新**：
$$h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)$$

**输出计算**（可选）：
$$y_t = W_{hy} \cdot h_t + b_y$$

### 1.2 符号说明

| 符号 | 含义 | 维度 |
|------|------|------|
| $x_t$ | 时间步 $t$ 的输入 | (input_size,) |
| $h_t$ | 时间步 $t$ 的隐状态 | (hidden_size,) |
| $h_{t-1}$ | 上一时间步的隐状态 | (hidden_size,) |
| $y_t$ | 时间步 $t$ 的输出 | (output_size,) |
| $W_{xh}$ | 输入到隐层的权重 | (hidden_size, input_size) |
| $W_{hh}$ | 隐层到隐层的权重 | (hidden_size, hidden_size) |
| $W_{hy}$ | 隐层到输出的权重 | (output_size, hidden_size) |
| $b_h$ | 隐层偏置 | (hidden_size,) |
| $b_y$ | 输出偏置 | (output_size,) |

In [ ]:
# ============================================================
# 维度追踪器：理解 RNN 的矩阵运算
# ============================================================

print("RNN 维度追踪器")
print("=" * 60)

# 定义维度
batch_size = 4      # 批次大小
seq_len = 5         # 序列长度
input_size = 10     # 输入特征维度
hidden_size = 20    # 隐层维度
output_size = 3     # 输出维度

print(f"\n配置：")
print(f"  batch_size  = {batch_size}")
print(f"  seq_len     = {seq_len}")
print(f"  input_size  = {input_size}")
print(f"  hidden_size = {hidden_size}")
print(f"  output_size = {output_size}")

print("\n" + "-" * 60)
print("维度追踪（单个时间步）：")
print("-" * 60)

# 创建示例数据
x_t = np.random.randn(batch_size, input_size)     # 当前输入
h_prev = np.random.randn(batch_size, hidden_size)  # 上一时刻隐状态

# 权重矩阵
W_xh = np.random.randn(hidden_size, input_size)   # 输入权重
W_hh = np.random.randn(hidden_size, hidden_size)  # 隐层权重
b_h = np.zeros(hidden_size)                        # 偏置

print(f"\n输入:")
print(f"  x_t    : {x_t.shape}    # (batch, input_size)")
print(f"  h_prev : {h_prev.shape} # (batch, hidden_size)")

print(f"\n权重:")
print(f"  W_xh   : {W_xh.shape} # (hidden_size, input_size)")
print(f"  W_hh   : {W_hh.shape} # (hidden_size, hidden_size)")
print(f"  b_h    : {b_h.shape}    # (hidden_size,)")

# 计算新的隐状态
# h_t = tanh(x_t @ W_xh.T + h_prev @ W_hh.T + b_h)
print(f"\n计算过程:")

step1 = x_t @ W_xh.T  # (batch, input) @ (input, hidden) = (batch, hidden)
print(f"  x_t @ W_xh.T    : {step1.shape}  # 输入变换")

step2 = h_prev @ W_hh.T  # (batch, hidden) @ (hidden, hidden) = (batch, hidden)
print(f"  h_prev @ W_hh.T : {step2.shape}  # 隐状态变换")

step3 = step1 + step2 + b_h  # 相加
print(f"  求和 + 偏置      : {step3.shape}  # 线性组合")

h_t = np.tanh(step3)  # 激活
print(f"  tanh(.)         : {h_t.shape}  # 新的隐状态")

---

## 💡 Part 2: NumPy 从零实现

### 2.1 实现 RNNCell

首先实现单个 RNN 单元，处理一个时间步。

In [ ]:
# ============================================================
# RNNCell: 单个 RNN 单元（NumPy 实现）
# ============================================================

class RNNCell:
    """
    RNN 单元 - 处理单个时间步
    
    数学公式:
        h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)
    
    参数:
        input_size: 输入特征维度
        hidden_size: 隐层维度
    
    属性:
        W_xh: 输入权重, shape (hidden_size, input_size)
        W_hh: 隐层权重, shape (hidden_size, hidden_size)
        b_h:  偏置, shape (hidden_size,)
    """
    
    def __init__(self, input_size, hidden_size):
        """
        初始化 RNN 单元
        
        使用 Xavier 初始化：std = sqrt(2 / (fan_in + fan_out))
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # ============================================================
        # 初始化权重
        # 使用较小的随机值，避免梯度爆炸
        # ============================================================
        
        # W_xh: 输入到隐层
        # shape: (hidden_size, input_size)
        scale_xh = np.sqrt(2.0 / (input_size + hidden_size))
        self.W_xh = np.random.randn(hidden_size, input_size) * scale_xh
        
        # W_hh: 隐层到隐层
        # shape: (hidden_size, hidden_size)
        scale_hh = np.sqrt(2.0 / (hidden_size + hidden_size))
        self.W_hh = np.random.randn(hidden_size, hidden_size) * scale_hh
        
        # b_h: 偏置
        # shape: (hidden_size,)
        self.b_h = np.zeros(hidden_size)
        
        print(f"RNNCell 初始化完成:")
        print(f"  input_size  = {input_size}")
        print(f"  hidden_size = {hidden_size}")
        print(f"  参数量: {self.count_params():,}")
    
    def forward(self, x_t, h_prev):
        """
        前向传播：处理单个时间步
        
        参数:
            x_t: 当前时间步的输入, shape (batch_size, input_size)
            h_prev: 上一时间步的隐状态, shape (batch_size, hidden_size)
        
        返回:
            h_t: 当前时间步的隐状态, shape (batch_size, hidden_size)
        """
        # ============================================================
        # 第1步：计算输入变换
        # x_t @ W_xh.T: (batch, input) @ (input, hidden) = (batch, hidden)
        # ============================================================
        input_transform = x_t @ self.W_xh.T
        
        # ============================================================
        # 第2步：计算隐状态变换
        # h_prev @ W_hh.T: (batch, hidden) @ (hidden, hidden) = (batch, hidden)
        # ============================================================
        hidden_transform = h_prev @ self.W_hh.T
        
        # ============================================================
        # 第3步：线性组合 + 偏置
        # ============================================================
        pre_activation = input_transform + hidden_transform + self.b_h
        
        # ============================================================
        # 第4步：激活函数 tanh
        # tanh 将输出压缩到 [-1, 1]，适合隐状态
        # ============================================================
        h_t = np.tanh(pre_activation)
        
        return h_t
    
    def count_params(self):
        """统计参数量"""
        return (self.W_xh.size + self.W_hh.size + self.b_h.size)
    
    def __call__(self, x_t, h_prev):
        """允许像函数一样调用"""
        return self.forward(x_t, h_prev)

In [ ]:
# ============================================================
# 测试 RNNCell
# ============================================================

print("测试 RNNCell")
print("=" * 50)

# 创建 RNNCell
input_size = 10
hidden_size = 20
batch_size = 4

cell = RNNCell(input_size, hidden_size)

# 创建测试数据
x_t = np.random.randn(batch_size, input_size)
h_prev = np.zeros((batch_size, hidden_size))  # 初始隐状态为零

print(f"\n输入:")
print(f"  x_t    : {x_t.shape}")
print(f"  h_prev : {h_prev.shape}")

# 前向传播
h_t = cell(x_t, h_prev)

print(f"\n输出:")
print(f"  h_t    : {h_t.shape}")
print(f"  h_t 范围: [{h_t.min():.3f}, {h_t.max():.3f}] (应在 [-1, 1])")

### 2.2 实现完整的 RNN

接下来实现完整的 RNN 类，可以处理整个序列。

In [ ]:
# ============================================================
# RNN: 完整的循环神经网络（NumPy 实现）
# ============================================================

class RNN:
    """
    完整的 RNN 网络 - 处理整个序列
    
    对于输入序列 x = [x_1, x_2, ..., x_T]，依次计算：
        h_1 = tanh(W_xh @ x_1 + W_hh @ h_0 + b_h)
        h_2 = tanh(W_xh @ x_2 + W_hh @ h_1 + b_h)
        ...
        h_T = tanh(W_xh @ x_T + W_hh @ h_{T-1} + b_h)
    
    参数:
        input_size: 输入特征维度
        hidden_size: 隐层维度
        batch_first: 输入格式是否为 (batch, seq, feature)
    """
    
    def __init__(self, input_size, hidden_size, batch_first=True):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        
        # 使用 RNNCell 作为核心计算单元
        self.cell = RNNCell(input_size, hidden_size)
    
    def forward(self, x, h_0=None):
        """
        前向传播：处理整个序列
        
        参数:
            x: 输入序列
               如果 batch_first=True: shape (batch_size, seq_len, input_size)
               如果 batch_first=False: shape (seq_len, batch_size, input_size)
            h_0: 初始隐状态, shape (batch_size, hidden_size)
                 如果为 None，则初始化为零
        
        返回:
            output: 所有时间步的隐状态
                    如果 batch_first=True: shape (batch_size, seq_len, hidden_size)
            h_n: 最后一个时间步的隐状态, shape (batch_size, hidden_size)
        """
        # ============================================================
        # 第1步：处理输入格式
        # ============================================================
        if self.batch_first:
            batch_size, seq_len, _ = x.shape
        else:
            seq_len, batch_size, _ = x.shape
            x = x.transpose(1, 0, 2)  # 转换为 (batch, seq, feature)
        
        # ============================================================
        # 第2步：初始化隐状态
        # ============================================================
        if h_0 is None:
            h_0 = np.zeros((batch_size, self.hidden_size))
        
        # ============================================================
        # 第3步：逐时间步计算
        # ============================================================
        h_t = h_0
        outputs = []  # 存储所有时间步的隐状态
        
        for t in range(seq_len):
            # 获取当前时间步的输入
            # x_t: (batch_size, input_size)
            x_t = x[:, t, :]
            
            # 计算当前时间步的隐状态
            h_t = self.cell(x_t, h_t)
            
            # 保存输出
            outputs.append(h_t)
        
        # ============================================================
        # 第4步：组装输出
        # ============================================================
        # output: (batch_size, seq_len, hidden_size)
        output = np.stack(outputs, axis=1)
        
        # h_n: 最后一个隐状态
        h_n = h_t
        
        if not self.batch_first:
            output = output.transpose(1, 0, 2)
        
        return output, h_n
    
    def __call__(self, x, h_0=None):
        return self.forward(x, h_0)

In [ ]:
# ============================================================
# 测试完整的 RNN
# ============================================================

print("测试完整的 RNN")
print("=" * 50)

# 配置
batch_size = 4
seq_len = 5
input_size = 10
hidden_size = 20

# 创建 RNN
rnn = RNN(input_size, hidden_size, batch_first=True)

# 创建输入序列
x = np.random.randn(batch_size, seq_len, input_size)

print(f"\n输入:")
print(f"  x : {x.shape}  # (batch, seq_len, input_size)")

# 前向传播
output, h_n = rnn(x)

print(f"\n输出:")
print(f"  output : {output.shape}  # (batch, seq_len, hidden_size)")
print(f"  h_n    : {h_n.shape}     # (batch, hidden_size)")

# 验证 h_n 是最后一个时间步的输出
print(f"\n验证 h_n == output[:, -1, :]:")
print(f"  差异最大值: {np.abs(h_n - output[:, -1, :]).max():.2e}")

---

## 💡 Part 3: 与 PyTorch 对比验证

为了验证我们的实现是正确的，我们将与 PyTorch 的 `nn.RNN` 进行对比。

In [ ]:
# ============================================================
# 与 PyTorch 的 nn.RNN 对比验证
# ============================================================

print("与 PyTorch nn.RNN 对比验证")
print("=" * 50)

# 配置
batch_size = 2
seq_len = 3
input_size = 4
hidden_size = 5

# 创建相同的输入
np.random.seed(123)
x_np = np.random.randn(batch_size, seq_len, input_size).astype(np.float32)
x_torch = torch.from_numpy(x_np)

# 创建 PyTorch RNN
torch_rnn = nn.RNN(input_size, hidden_size, batch_first=True, nonlinearity='tanh')

# 创建我们的 RNN，并使用相同的权重
our_rnn = RNN(input_size, hidden_size, batch_first=True)

# ============================================================
# 关键：复制 PyTorch 的权重到我们的实现
# PyTorch 的 RNN 权重命名：
#   weight_ih_l0: 输入权重 (hidden_size, input_size)
#   weight_hh_l0: 隐层权重 (hidden_size, hidden_size)
#   bias_ih_l0 + bias_hh_l0: 偏置
# ============================================================
our_rnn.cell.W_xh = torch_rnn.weight_ih_l0.detach().numpy()
our_rnn.cell.W_hh = torch_rnn.weight_hh_l0.detach().numpy()
# PyTorch 有两个偏置，需要相加
our_rnn.cell.b_h = (torch_rnn.bias_ih_l0.detach().numpy() + 
                    torch_rnn.bias_hh_l0.detach().numpy())

# 初始隐状态
h_0_np = np.zeros((1, batch_size, hidden_size), dtype=np.float32)
h_0_torch = torch.from_numpy(h_0_np)

# 前向传播
with torch.no_grad():
    output_torch, h_n_torch = torch_rnn(x_torch, h_0_torch)

output_np, h_n_np = our_rnn(x_np, h_0_np.squeeze(0))

# 比较结果
output_torch_np = output_torch.numpy()
h_n_torch_np = h_n_torch.squeeze(0).numpy()

print(f"\n输出对比:")
print(f"  PyTorch output shape : {output_torch.shape}")
print(f"  Our output shape     : {output_np.shape}")

output_diff = np.abs(output_np - output_torch_np).max()
h_n_diff = np.abs(h_n_np - h_n_torch_np).max()

print(f"\n数值差异:")
print(f"  output 最大差异: {output_diff:.2e}")
print(f"  h_n 最大差异   : {h_n_diff:.2e}")

if output_diff < 1e-5 and h_n_diff < 1e-5:
    print(f"\n✅ 验证通过！我们的实现与 PyTorch 一致！")
else:
    print(f"\n❌ 验证失败，存在差异！")

---

## 💡 Part 4: 可视化隐状态演变

让我们可视化 RNN 处理序列时隐状态的变化。

In [ ]:
# ============================================================
# 可视化：隐状态随时间步的演变
# ============================================================

# 创建一个较长的序列
seq_len = 20
input_size = 5
hidden_size = 10

rnn = RNN(input_size, hidden_size)

# 创建一个有规律的输入（便于观察）
t = np.linspace(0, 4*np.pi, seq_len)
x = np.zeros((1, seq_len, input_size))
for i in range(input_size):
    x[0, :, i] = np.sin(t + i * np.pi / input_size)

# 前向传播
output, h_n = rnn(x)

# 可视化
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 输入序列
ax1 = axes[0]
im1 = ax1.imshow(x[0].T, aspect='auto', cmap='RdBu_r')
ax1.set_xlabel('时间步', fontsize=12)
ax1.set_ylabel('输入维度', fontsize=12)
ax1.set_title('输入序列 x', fontsize=14)
plt.colorbar(im1, ax=ax1)

# 隐状态演变
ax2 = axes[1]
im2 = ax2.imshow(output[0].T, aspect='auto', cmap='RdBu_r')
ax2.set_xlabel('时间步', fontsize=12)
ax2.set_ylabel('隐状态维度', fontsize=12)
ax2.set_title('隐状态演变 h', fontsize=14)
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

print("\n💡 观察：")
print("   - 输入是周期性的正弦波")
print("   - 隐状态随时间步不断更新")
print("   - 隐状态'记住'了之前的输入信息")

In [ ]:
# ============================================================
# 可视化：不同输入对隐状态的影响
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 四种不同的输入模式
patterns = [
    ("全零输入", np.zeros((1, seq_len, input_size))),
    ("随机输入", np.random.randn(1, seq_len, input_size)),
    ("递增输入", np.linspace(0, 1, seq_len).reshape(1, -1, 1).repeat(input_size, axis=2)),
    ("脉冲输入", np.eye(seq_len, input_size).reshape(1, seq_len, input_size)),
]

for ax, (name, x_pattern) in zip(axes.flat, patterns):
    output, _ = rnn(x_pattern)
    im = ax.imshow(output[0].T, aspect='auto', cmap='RdBu_r', vmin=-1, vmax=1)
    ax.set_xlabel('时间步')
    ax.set_ylabel('隐状态维度')
    ax.set_title(f'{name}的隐状态演变')
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

print("\n💡 观察：")
print("   - 全零输入：隐状态保持不变（接近零）")
print("   - 随机输入：隐状态随机波动")
print("   - 递增输入：隐状态逐渐趋于稳定")
print("   - 脉冲输入：隐状态对每个脉冲有响应")

---

## 💡 Part 5: 简单任务验证

让我们用一个简单的任务来验证 RNN 的序列建模能力：**序列求和**。

任务：给定一个数字序列，预测序列的累积和。

In [ ]:
# ============================================================
# 简单任务：序列累积求和
# ============================================================

print("简单任务：序列累积求和")
print("=" * 50)

# 生成数据
def generate_cumsum_data(n_samples, seq_len):
    """
    生成累积求和数据
    
    输入: 随机数序列 [x_1, x_2, ..., x_T]
    输出: 累积和序列 [x_1, x_1+x_2, ..., x_1+...+x_T]
    """
    X = np.random.randn(n_samples, seq_len, 1).astype(np.float32)
    Y = np.cumsum(X, axis=1)
    return X, Y

# 生成训练数据
n_samples = 1000
seq_len = 10
X_train, Y_train = generate_cumsum_data(n_samples, seq_len)

print(f"训练数据：")
print(f"  X_train: {X_train.shape}")
print(f"  Y_train: {Y_train.shape}")

# 示例
print(f"\n示例：")
print(f"  输入: {X_train[0, :, 0].round(2)}")
print(f"  输出: {Y_train[0, :, 0].round(2)}")

In [ ]:
# ============================================================
# 使用 PyTorch 训练 RNN 完成累积求和任务
# ============================================================

class SimpleRNN(nn.Module):
    """用于累积求和任务的简单 RNN"""
    
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # RNN 输出
        output, _ = self.rnn(x)
        # 全连接层
        output = self.fc(output)
        return output

# 创建模型
model = SimpleRNN(input_size=1, hidden_size=32, output_size=1)

# 优化器和损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# 转换数据
X_tensor = torch.from_numpy(X_train)
Y_tensor = torch.from_numpy(Y_train)

# 训练
n_epochs = 100
batch_size = 64
losses = []

print("\n训练 RNN...")
for epoch in range(n_epochs):
    # 随机采样一个批次
    idx = np.random.choice(n_samples, batch_size, replace=False)
    X_batch = X_tensor[idx]
    Y_batch = Y_tensor[idx]
    
    # 前向传播
    output = model(X_batch)
    loss = criterion(output, Y_batch)
    
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"  Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

print("\n训练完成！")

In [ ]:
# ============================================================
# 可视化训练结果
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线
ax1 = axes[0]
ax1.plot(losses, 'b-', linewidth=1)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('训练损失曲线', fontsize=14)
ax1.grid(True, alpha=0.3)

# 预测 vs 真实值
ax2 = axes[1]

# 测试一个样本
model.eval()
with torch.no_grad():
    test_x = X_tensor[0:1]
    test_y = Y_tensor[0:1]
    pred_y = model(test_x)

time_steps = np.arange(seq_len)
ax2.plot(time_steps, test_y[0, :, 0].numpy(), 'b-o', linewidth=2, label='真实值')
ax2.plot(time_steps, pred_y[0, :, 0].numpy(), 'r--s', linewidth=2, label='预测值')
ax2.set_xlabel('时间步', fontsize=12)
ax2.set_ylabel('累积和', fontsize=12)
ax2.set_title('累积求和：预测 vs 真实', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 结论：")
print("   - RNN 成功学会了累积求和任务")
print("   - 这证明 RNN 能够'记住'之前的输入并累积信息")
print("   - 这正是 RNN 的核心能力：序列建模")

---

## 📝 本章小结

### 核心公式

$$h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)$$

### 关键实现

1. **RNNCell**：处理单个时间步
   - 输入变换：$W_{xh} \cdot x_t$
   - 隐状态变换：$W_{hh} \cdot h_{t-1}$
   - 激活函数：$\tanh$

2. **RNN**：处理整个序列
   - 循环调用 RNNCell
   - 收集所有时间步的输出

### 参数维度

| 参数 | 维度 |
|------|------|
| W_xh | (hidden_size, input_size) |
| W_hh | (hidden_size, hidden_size) |
| b_h  | (hidden_size,) |

---

## 🎯 练习题

### 练习 1: 添加输出层

修改 RNNCell 类，添加输出计算：$y_t = W_{hy} \cdot h_t + b_y$

In [ ]:
# 练习 1 答案区域
# ============================================================

class RNNCellWithOutput:
    """带输出层的 RNN 单元"""
    
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 隐层权重
        scale_xh = np.sqrt(2.0 / (input_size + hidden_size))
        self.W_xh = np.random.randn(hidden_size, input_size) * scale_xh
        scale_hh = np.sqrt(2.0 / (hidden_size + hidden_size))
        self.W_hh = np.random.randn(hidden_size, hidden_size) * scale_hh
        self.b_h = np.zeros(hidden_size)
        
        # 输出层权重
        scale_hy = np.sqrt(2.0 / (hidden_size + output_size))
        self.W_hy = np.random.randn(output_size, hidden_size) * scale_hy
        self.b_y = np.zeros(output_size)
    
    def forward(self, x_t, h_prev):
        # 隐状态更新
        h_t = np.tanh(x_t @ self.W_xh.T + h_prev @ self.W_hh.T + self.b_h)
        # 输出计算
        y_t = h_t @ self.W_hy.T + self.b_y
        return h_t, y_t

# 测试
cell = RNNCellWithOutput(input_size=10, hidden_size=20, output_size=5)
x_t = np.random.randn(4, 10)
h_prev = np.zeros((4, 20))
h_t, y_t = cell.forward(x_t, h_prev)
print(f"h_t shape: {h_t.shape}")
print(f"y_t shape: {y_t.shape}")

### 练习 2: 实现双向 RNN 的前向传播

实现双向 RNN：同时从前向后和从后向前处理序列。

In [ ]:
# 练习 2 答案区域
# ============================================================

class BiRNN:
    """双向 RNN"""
    
    def __init__(self, input_size, hidden_size):
        # 前向 RNN
        self.forward_rnn = RNN(input_size, hidden_size)
        # 后向 RNN
        self.backward_rnn = RNN(input_size, hidden_size)
    
    def forward(self, x):
        """
        x: (batch, seq_len, input_size)
        返回: (batch, seq_len, hidden_size * 2)
        """
        # 前向传播
        output_forward, _ = self.forward_rnn(x)
        
        # 后向传播（先反转序列）
        x_reversed = x[:, ::-1, :].copy()  # 反转序列
        output_backward, _ = self.backward_rnn(x_reversed)
        output_backward = output_backward[:, ::-1, :]  # 再反转回来
        
        # 拼接前向和后向的输出
        output = np.concatenate([output_forward, output_backward], axis=-1)
        
        return output

# 测试
birnn = BiRNN(input_size=10, hidden_size=20)
x = np.random.randn(4, 5, 10)
output = birnn.forward(x)
print(f"输入: {x.shape}")
print(f"输出: {output.shape}  # 隐层维度翻倍")

---

## 🚀 下一步

现在你已经能够从零实现 RNN，接下来我们将学习：

👉 [03_bptt_algorithm.ipynb](./03_bptt_algorithm.ipynb) - BPTT 反向传播算法

在下一章，你将：
- 推导 BPTT（BackPropagation Through Time）算法
- 用 NumPy 实现 RNN 的反向传播
- 理解梯度如何在时间维度上传播
- 实现梯度裁剪和数值梯度验证